In [1]:
import copy
import pprint
import torch
import torchio as tio
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import os.path
from os import path
sns.set_style("whitegrid", {'axes.grid' : False})
%config InlineBackend.figure_format = 'retina'
torch.manual_seed(14041931)
print('TorchIO version:', tio.__version__)

from torch.utils.data import DataLoader

TorchIO version: 0.18.39


## Load an anatomical image and its respective masks

In [2]:
sujet_index = 1
path_database = "multiclass-segmentation/Data_SC_GM_WM"
image = tio.ScalarImage(path_database + "/Sub-" + str(sujet_index).zfill(2) + '_'+'T2.nii.gz')
label_SC = tio.LabelMap(path_database + "/Sub-" + str(sujet_index).zfill(2) + '_' + 'SC.nii.gz')
label_GM  = tio.LabelMap(path_database + "/Sub-" + str(sujet_index).zfill(2) + '_'+'GM.nii.gz')

## Creation of a grid to observe the deformations

In [3]:
grid = copy.deepcopy(image)
data = grid.data
white = data.max()
N = 7
data[..., ::N, :, :] = white
data[..., :, ::N, :] = white


## Definition of the transformations to be carried out

In [6]:
Subjects={"image":image,"label_SC":label_SC,"label_GM":label_GM,"grid":grid}
Subjects=tio.Subject(Subjects)

max_displacement = 3.5, 4.5, 0  # in x, y and z directions
#transforms = [tio.RandomElasticDeformation(max_displacement=max_displacement),tio.RandomAffine(scales=(1.1, 1.1))]
transforms = [tio.RandomElasticDeformation(max_displacement=max_displacement,image_interpolation='linear'),
              tio.RandomAffine(scales=(1.1, 1.1),degrees=5,isotropic=False, image_interpolation='linear')]
transform = tio.Compose(transforms)
transformed_Aff = transform(Subjects)


transforms_Ghost = [tio.RandomGhosting(num_ghosts=3,axes=(0,1),intensity=0.8)]
transform_Ghost = tio.Compose(transforms_Ghost)
transformed_Ghost = transform_Ghost(Subjects)



## Output of transformed images (data augmentation)

In [5]:
path_database_out="Transformations"
save_image =(path_database_out + '/B_'+'Aff_T2.nii.gz')
save_label_GM =(path_database_out + '/B_'+'Aff_GM.nii.gz')
save_label_SC =(path_database_out + '/B_'+'Aff_SC.nii.gz')
save_grid =(path_database_out +  '/B_'+'Aff_Grid.nii.gz')

save_image_Ghost =(path_database_out + '/B_'+'Ghost_T2.nii.gz')
save_label_SC_Ghost =(path_database_out +'/B_'+'Ghost_SC.nii.gz')
save_label_GM_Ghost =(path_database_out +  '/B_'+'Ghost_GM.nii.gz')

(transformed_Aff["image"]).save(save_image)
(transformed_Aff["label_SC"]).save(save_label_SC)
(transformed_Aff["label_GM"]).save(save_label_GM)
(transformed_Aff["grid"]).save(save_grid)


(transformed_Ghost["image"]).save(save_image_Ghost)
(transformed_Ghost["label_SC"]).save(save_label_SC_Ghost)
(transformed_Ghost["label_GM"]).save(save_label_GM_Ghost)